# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'04-06-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'04-06-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-04-07 04:20:53,33.93911,67.709953,56779,2512,51928.0,2339.0,Afghanistan,145.855175,4.424171
1,NaN,NaN,NaN,Albania,2021-04-07 04:20:53,41.15330,20.168300,126936,2283,95035.0,29618.0,Albania,4410.869414,1.798544
2,NaN,NaN,NaN,Algeria,2021-04-07 04:20:53,28.03390,1.659600,117879,3112,82096.0,32671.0,Algeria,268.816867,2.639995
3,NaN,NaN,NaN,Andorra,2021-04-07 04:20:53,42.50630,1.521800,12328,117,11570.0,641.0,Andorra,15955.477901,0.949059
4,NaN,NaN,NaN,Angola,2021-04-07 04:20:53,-11.20270,17.873900,22885,547,21489.0,849.0,Angola,69.630662,2.390212


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,56294,56322,56384,56454,56517,56572,56595,56676,56717,56779
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,124134,124419,124723,125157,125506,125842,126183,126531,126795,126936
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,116836,116946,117061,117192,117304,117429,117524,117622,117739,117879


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2470,2472,2476,2484,2489,2495,2496,2497,2508,2512
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2210,2216,2227,2235,2241,2247,2256,2265,2274,2283
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3080,3084,3089,3093,3096,3099,3102,3105,3108,3112


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,50013,50666,51473,51550,51788,51798,51802,51885,51902,51928
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,89456,90024,90617,91271,91875,92500,93173,93842,94431,95035
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,81316,81349,81442,81538,81632,81729,81813,81896,81994,82096


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6570,6577,6580,6589,6595,6606,6617,6619,6620,6644
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,20473,20487,20492,20505,20523,20519,20526,20541,20542,20551
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2226,2226,2227,2227,2227,2228,2231,2232,2232,2238


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,99,99,99,99,99,99,102,102,102,102
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,301,301,301,301,301,301,301,301,301,301
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,54,54,54,55,55,55,55,55,55,55


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-04-07 04:20:53,32.539527,-86.644082,6644,102,NaN,NaN,"Autauga, Alabama, US",11892.104745,1.535220
689,1075.0,Lamar,Alabama,US,2021-04-07 04:20:53,33.779950,-88.096680,1392,33,NaN,NaN,"Lamar, Alabama, US",10083.303151,2.370690
690,1077.0,Lauderdale,Alabama,US,2021-04-07 04:20:53,34.901719,-87.656247,9165,227,NaN,NaN,"Lauderdale, Alabama, US",9883.639422,2.476814


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,30846396,556509,0.0
Brazil,13100580,336947,11459548.0
India,12686049,165547,11732279.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,30846396,556509,0.0,30289887.0,2021-04-07 04:20:53,37.950547,-91.419547
Brazil,13100580,336947,11459548.0,1304085.0,2021-04-07 04:20:53,-12.669522,-48.480493
India,12686049,165547,11732279.0,788223.0,2021-04-07 04:20:53,23.088275,81.806127
France,4902025,97431,308128.0,4496466.0,2021-04-07 04:20:53,6.430808,-34.730285
Russia,4546307,99431,4171948.0,274928.0,2021-04-07 04:20:53,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3684276,59868,0.0
Texas,2811270,48815,0.0
Florida,2090862,33780,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3684276,59868,0.0,3624408.0,2021-04-07 04:20:53,37.843962,-120.728594
Texas,2811270,48815,0.0,2762455.0,2021-04-07 04:20:53,31.660643,-98.653069
Florida,2090862,33780,0.0,2057082.0,2021-04-07 04:20:53,28.940755,-82.700744
New York,1918437,50751,0.0,1867686.0,2021-04-07 04:20:53,42.544151,-75.474183
Illinois,1261577,23673,0.0,1237904.0,2021-04-07 04:20:53,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1222778,23302,0.0
Arizona,Maricopa,526415,9670,0.0
Illinois,Cook,505380,9709,0.0
Florida,Miami-Dade,453345,5924,0.0
Texas,Harris,380297,5888,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1222778,23302,0.0,1199476.0,2021-04-07 04:20:53,34.308284,-118.228241,6037.0
Arizona,Maricopa,526415,9670,0.0,516745.0,2021-04-07 04:20:53,33.348359,-112.491815,4013.0
Illinois,Cook,505380,9709,0.0,495671.0,2021-04-07 04:20:53,41.841448,-87.816588,17031.0
Florida,Miami-Dade,453345,5924,0.0,447421.0,2021-04-07 04:20:53,25.611236,-80.551706,12086.0
Texas,Harris,380297,5888,0.0,374409.0,2021-04-07 04:20:53,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-04,56676,126531,117622,12231,22631,1170,2393492,196634,29357,558755,...,4373798,117757,83623,3,166123,2631,251288,4798,88930,36923
2021-04-05,56717,126795,117739,12286,22717,1173,2407159,197113,29365,560972,...,4376629,119958,83802,3,167548,2637,253922,4881,89009,36934
2021-04-06,56779,126936,117879,12328,22885,1173,2428029,197873,29379,562907,...,4379033,123063,83935,3,169074,2648,256461,4975,89071,36966


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-04,2497,2265,3105,117,542,28,56199,3592,909,9449,...,127078,1101,631,0,1662,35,2706,946,1220,1525
2021-04-05,2508,2274,3108,117,543,29,56471,3614,909,9482,...,127106,1146,631,0,1678,35,2716,955,1222,1525
2021-04-06,2512,2283,3112,117,547,29,56634,3627,909,9517,...,127126,1191,632,0,1693,35,2735,976,1224,1531


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-04,51885,93842,81896,11474,20879,882,2140480,176009,23032,512642,...,13254,89035,81629,1,150374,2383,222007,1738,85068,34748
2021-04-05,51902,94431,81994,11523,21452,885,2153509,176889,23048,516084,...,13296,94042,81763,1,152201,2416,223249,1772,85178,34758
2021-04-06,51928,95035,82096,11570,21489,885,2164045,177601,23049,519202,...,13345,96563,81881,1,153063,2422,224552,1822,85338,34781


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6570,6577,6580,6589,6595,6606,6617,6619,6620,6644
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1384,1384,1384,1384,1390,1392,1392,1392,1392,1392
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9100,9102,9116,9140,9144,9148,9156,9159,9162,9165
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2918,2922,2926,2926,2928,2931,2932,2932,2932,2938
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15389,15394,15397,15417,15433,15444,15452,15457,15459,15468


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-04,516809,63501,0,844302,331054,3676742,468121,313956,96376,49,...,813614,2802225,387341,20149,2907,625148,368403,143456,639227,56447
2021-04-05,516918,64001,0,844910,331098,3682861,468988,316655,96577,49,...,817022,2806908,387514,20267,2945,626171,370017,143733,639553,56529
2021-04-06,517114,64081,0,845480,331261,3684276,470254,317729,96806,49,...,818008,2811270,387814,20373,2945,627605,370652,144010,640704,56619


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-04-04        6619   20541    2232  2546   6459    1209   2106   14249   
2021-04-05        6620   20542    2232  2546   6460    1209   2107   14251   
2021-04-06        6644   20551    2238  2549   6462    1209   2113   14245   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-04         3493     1812  ...    2641    611     3083      710   
2021-04-05         3493     1812  ...    2644    611     3084      717   
2021-04-06         3494     1814  ...    2644    611     3084      723   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-04           4037  3624  2139          0      891    634  
2021-04-05           4053  3632  2134          0      892    635  
2021-04-06           4060  3640  2135          0      892    635  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-04,10637,313,0,16990,5643,59711,6126,7904,1566,0,...,11915,48697,2132,229,26,10329,5278,2695,7340,700
2021-04-05,10638,313,0,16990,5648,59799,6126,7923,1566,0,...,11929,48729,2133,229,26,10360,5285,2696,7341,700
2021-04-06,10652,313,0,16996,5653,59868,6132,7930,1567,0,...,11967,48815,2137,229,26,10401,5299,2707,7349,701


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-04-04         102     301      55   58    132      40     66     310   
2021-04-05         102     301      55   58    132      40     66     310   
2021-04-06         102     301      55   58    132      40     66     310   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-04          120       43  ...      30     11       31        7   
2021-04-05          120       43  ...      30     11       31        7   
2021-04-06          121       43  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-04             37     9    12          0       26      5  
2021-04-05             37     9    12          0       26      5  
2021-04-06             37     9    12          0       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-04,0.001431,0.002758,0.000834,0.004682,0.002303,0.000000,0.004177,0.003010,0.000307,0.004933,...,0.000550,0.033827,0.002253,0.0,0.010868,0.001904,0.011293,0.021503,0.001464,0.000325
2021-04-05,0.000723,0.002086,0.000995,0.004497,0.003800,0.002564,0.005710,0.002436,0.000273,0.003968,...,0.000647,0.018691,0.002141,0.0,0.008578,0.002281,0.010482,0.017299,0.000888,0.000298
2021-04-06,0.001093,0.001112,0.001189,0.003419,0.007395,0.000000,0.008670,0.003856,0.000477,0.003449,...,0.000549,0.025884,0.001587,0.0,0.009108,0.004171,0.009999,0.019258,0.000697,0.000866


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-04,0.000401,0.003989,0.000967,0.0,0.003704,0.000000,0.001658,0.004755,0.0,0.001590,...,0.000079,0.028011,0.001587,NaN,0.009107,0.0,0.009325,0.015021,0.004115,0.000656
2021-04-05,0.004405,0.003974,0.000966,0.0,0.001845,0.035714,0.004840,0.006125,0.0,0.003492,...,0.000220,0.040872,0.000000,NaN,0.009627,0.0,0.003695,0.009514,0.001639,0.000000
2021-04-06,0.001595,0.003958,0.001287,0.0,0.007366,0.000000,0.002886,0.003597,0.0,0.003691,...,0.000157,0.039267,0.001585,NaN,0.008939,0.0,0.006996,0.021990,0.001637,0.003934


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-04,0.001602,0.007180,0.001015,0.004025,0.000383,0.000000,0.004618,0.003466,0.000217,0.006068,...,0.002420,0.000000,0.002210,0.0,0.000000,0.000000,0.007209,0.013411,0.000600,0.000461
2021-04-05,0.000328,0.006277,0.001197,0.004271,0.027444,0.003401,0.006087,0.005000,0.000695,0.006714,...,0.003169,0.056236,0.001642,0.0,0.012150,0.013848,0.005594,0.019563,0.001293,0.000288
2021-04-06,0.000501,0.006396,0.001244,0.004079,0.001725,0.000000,0.004892,0.004025,0.000043,0.006042,...,0.003685,0.026807,0.001443,0.0,0.005664,0.002483,0.005837,0.028217,0.001878,0.000662


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-04,0.000376,0.000000,NaN,0.000550,0.000254,0.000177,0.002613,0.000000,0.003112,0.0,...,0.000000,0.000359,0.000889,0.012258,0.000000,0.002031,0.000000,0.002446,0.000998,0.000000
2021-04-05,0.000211,0.007874,NaN,0.000720,0.000133,0.001664,0.001852,0.008597,0.002086,0.0,...,0.004189,0.001671,0.000447,0.005856,0.013072,0.001636,0.004381,0.001931,0.000510,0.001453
2021-04-06,0.000379,0.001250,NaN,0.000675,0.000492,0.000384,0.002699,0.003392,0.002371,0.0,...,0.001207,0.001554,0.000774,0.005230,0.000000,0.002290,0.001716,0.001927,0.001800,0.001592


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-04-04      0.000302  0.000731  0.000448  0.000393  0.000155  0.000828   
2021-04-05      0.000151  0.000049  0.000000  0.000000  0.000155  0.000000   
2021-04-06      0.003625  0.000438  0.002688  0.001178  0.000310  0.000000   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2021-04-04      0.000000  0.000421  0.000860  0.000000  ...  0.000000    0.0   
2021-04-05      0.000475  0.000140  0.000000  0.000000  ...  0.001136    0.0   
2021-04-06      0.002848 -0.000421  0.000286  0.001104  ...  0.000000    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2021-04-04      0.000000  0.000000   0.000000  0.000000  0.000000        NaN   
2021-04-05      0.000324  0.009859   0.003963  0.002208 -0.002338        NaN   
2021-04-06      0.000000  0.008368   0.001727  0.002203  0.000469        NaN   

Province_State                      
Admin2          Washakie    Weston  
2021-04-04      0.000000  0.000000  
2021-04-05      0.001122  0.001577  
2021-04-06      0.000000  0.000000  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-04,0.000000,0.0,NaN,-0.000059,0.000355,0.001023,0.000327,0.000000,0.000639,NaN,...,0.000000,0.000431,0.000000,0.0,0.0,0.004083,0.000000,0.000743,-0.000136,0.000000
2021-04-05,0.000094,0.0,NaN,0.000000,0.000886,0.001474,0.000000,0.002404,0.000000,NaN,...,0.001175,0.000657,0.000469,0.0,0.0,0.003001,0.001326,0.000371,0.000136,0.000000
2021-04-06,0.001316,0.0,NaN,0.000353,0.000885,0.001154,0.000979,0.000884,0.000639,NaN,...,0.003186,0.001765,0.001875,0.0,0.0,0.003958,0.002649,0.004080,0.001090,0.001429


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-04-04         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2021-04-05         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2021-04-06         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                     ... Wyoming                           \
Admin2          Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-04      0.000000      0.0  ...     0.0    0.0      0.0      0.0   
2021-04-05      0.000000      0.0  ...     0.0    0.0      0.0      0.0   
2021-04-06      0.008333      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-04            0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-05            0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-06            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-04,0.001071,0.002760,0.000876,0.004731,0.003282,0.005164,0.004451,0.004415,0.000303,0.005440,...,0.000695,0.029385,0.002270,7.450581e-09,0.009640,0.002132,0.009525,0.021486,0.001419,0.000307
2021-04-05,0.000897,0.002423,0.000935,0.004614,0.003541,0.003864,0.005080,0.003425,0.000288,0.004704,...,0.000671,0.024038,0.002205,3.725290e-09,0.009109,0.002206,0.010003,0.019393,0.001154,0.000302
2021-04-06,0.000995,0.001768,0.001062,0.004016,0.005468,0.001932,0.006875,0.003640,0.000382,0.004077,...,0.000610,0.024961,0.001896,1.862645e-09,0.009108,0.003189,0.010001,0.019325,0.000925,0.000584


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-04,0.000864,0.003740,0.000991,0.003261,0.003034,5.809583e-07,0.001617,0.005306,1.737579e-33,0.002559,...,0.000157,0.028799,0.001003,NaN,0.009395,1.510566e-66,0.008093,0.015336,0.002774,0.000477
2021-04-05,0.002635,0.003857,0.000978,0.001631,0.002440,1.785743e-02,0.003228,0.005715,8.687895e-34,0.003025,...,0.000189,0.034835,0.000502,NaN,0.009511,7.552828e-67,0.005894,0.012425,0.002207,0.000238
2021-04-06,0.002115,0.003907,0.001133,0.000815,0.004903,8.928717e-03,0.003057,0.004656,4.343948e-34,0.003358,...,0.000173,0.037051,0.001043,NaN,0.009225,3.776414e-67,0.006445,0.017207,0.001922,0.002086


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-04,0.001535,0.007101,0.001050,0.003479,0.002641,0.010393,0.004411,0.003987,0.000207,0.005913,...,0.003387,0.011286,0.002054,0.0,0.002892,0.001658,0.008331,0.011709,0.001193,0.000532
2021-04-05,0.000931,0.006689,0.001123,0.003875,0.015042,0.006897,0.005249,0.004493,0.000451,0.006314,...,0.003278,0.033761,0.001848,0.0,0.007521,0.007753,0.006963,0.015636,0.001243,0.000410
2021-04-06,0.000716,0.006543,0.001184,0.003977,0.008384,0.003449,0.005071,0.004259,0.000247,0.006178,...,0.003482,0.030284,0.001645,0.0,0.006592,0.005118,0.006400,0.021926,0.001561,0.000536


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-04,0.000557,0.000961,NaN,0.000701,0.000419,0.000368,0.002996,0.001198,0.003772,8.639802e-117,...,0.000234,0.000630,0.001018,0.010830,0.000030,0.002252,0.001743,0.002776,0.001137,0.000293
2021-04-05,0.000384,0.004418,NaN,0.000711,0.000276,0.001016,0.002424,0.004897,0.002929,4.319901e-117,...,0.002211,0.001151,0.000733,0.008343,0.006551,0.001944,0.003062,0.002354,0.000824,0.000873
2021-04-06,0.000382,0.002834,NaN,0.000693,0.000384,0.000700,0.002562,0.004145,0.002650,2.159950e-117,...,0.001709,0.001352,0.000753,0.006787,0.003276,0.002117,0.002389,0.002140,0.001312,0.001233


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-04-04      0.000903  0.000518  0.000632  0.000439  0.000442  0.000684   
2021-04-05      0.000527  0.000283  0.000316  0.000219  0.000299  0.000342   
2021-04-06      0.002076  0.000361  0.001502  0.000699  0.000304  0.000171   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-04-04      0.000369  0.000487  0.000542  0.000368  ...  0.000016   
2021-04-05      0.000422  0.000313  0.000271  0.000184  ...  0.000576   
2021-04-06      0.001635 -0.000054  0.000279  0.000644  ...  0.000288   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-04-04      0.000270  0.000201  0.000945   0.000514  0.000576  0.000546   
2021-04-05      0.000135  0.000263  0.005402   0.002239  0.001392 -0.000896   
2021-04-06      0.000068  0.000131  0.006885   0.001983  0.001797 -0.000214   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-04-04           -1.0  0.000009  0.000148  
2021-04-05           -1.0  0.000566  0.000863  
2021-04-06           -1.0  0.000283  0.000431  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-04,0.001427,2.396265e-06,NaN,0.000201,0.000559,0.001371,0.000594,0.000190,0.000923,NaN,...,0.000157,0.001256,0.000685,0.001952,0.000039,0.002563,0.000666,0.001693,0.000503,0.000225
2021-04-05,0.000760,1.198133e-06,NaN,0.000100,0.000723,0.001422,0.000297,0.001297,0.000461,NaN,...,0.000666,0.000957,0.000577,0.000976,0.000020,0.002782,0.000996,0.001032,0.000320,0.000113
2021-04-06,0.001038,5.990663e-07,NaN,0.000227,0.000804,0.001288,0.000638,0.001090,0.000550,NaN,...,0.001926,0.001361,0.001226,0.000488,0.000010,0.003370,0.001823,0.002556,0.000705,0.000771


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin   Barbour          Bibb    Blount   
2021-04-04      0.007577  0.000006  0.000579 -1.548286e-11  0.000956   
2021-04-05      0.003789  0.000003  0.000289 -7.741430e-12  0.000478   
2021-04-06      0.001894  0.000002  0.000145 -3.870715e-12  0.000239   

Province_State                                                        ...  \
Admin2           Bullock        Butler   Calhoun  Chambers  Cherokee  ...   
2021-04-04      0.000801  1.190451e-13  0.002152  0.004513  0.000012  ...   
2021-04-05      0.000401  5.952253e-14  0.001076  0.002257  0.000006  ...   
2021-04-06      0.000200  2.976126e-14  0.000538  0.005295  0.000003  ...   

Province_State   Wyoming                                                       \
Admin2              Park        Platte      Sheridan      Sublette Sweetwater   
2021-04-04      0.000004  1.708772e-22  3.179561e-08  1.396810e-28   0.000868   
2021-04-05      0.000002  8.543862e-23  1.589781e-08  6.984049e-29   0.000434   
2021-04-06      0.000001  4.271931e-23  7.948903e-09  3.492024e-29   0.000217   

Province_State                                                       \
Admin2                 Teton         Uinta Unassigned      Washakie   
2021-04-04      4.458880e-16  2.265975e-23  -0.501961  8.678614e-21   
2021-04-05      2.229440e-16  1.132988e-23  -0.501961  4.339307e-21   
2021-04-06      1.114720e-16  5.664938e-24  -0.501961  2.169654e-21   

Province_State                
Admin2                Weston  
2021-04-04      1.843144e-19  
2021-04-05      9.215719e-20  
2021-04-06      4.607859e-20  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
